### The Scenario

You are an analyst at a SaaS company. You've been given a raw table of user events, and your goal is to group these individual events into "sessions." A session is defined as a continuous period of activity by a user where the time between consecutive events does not exceed **30 minutes**.

### Database Schema and Sample Data

Here is the schema and sample data for our `events` table. The data is crafted to test several edge cases, like single-event sessions and back-to-back sessions for the same user.

```sql
-- Create the table
CREATE TABLE events (
    event_id INT PRIMARY KEY,
    user_id INT,
    event_timestamp TIMESTAMP,
    event_type VARCHAR(50)
);

-- Insert sample data
INSERT INTO events VALUES
-- User 1: Two distinct sessions
(1, 101, '2025-09-07 10:00:00', 'page_view'),
(2, 101, '2025-09-07 10:05:00', 'click'),
(3, 101, '2025-09-07 10:15:00', 'add_to_cart'),
(4, 101, '2025-09-07 11:00:00', 'page_view'), -- >30 mins later, new session
(5, 101, '2025-09-07 11:02:00', 'click'),

-- User 2: One long session
(6, 102, '2025-09-07 12:00:00', 'page_view'),
(7, 102, '2025-09-07 12:25:00', 'add_to_cart'),
(8, 102, '2025-09-07 12:45:00', 'purchase'),

-- User 3: A single event session
(9, 103, '2025-09-07 13:00:00', 'page_view');

```

*Note: The exact functions for timestamp differences may vary slightly by SQL dialect (e.g., `TIMESTAMPDIFF` in MySQL, subtracting timestamps in PostgreSQL).*

-----

### The Challenge: A Three-Part Query

Your task is to build a single query that generates a summary of each user session. This is best done by building up the logic step-by-step using Common Table Expressions (CTEs).

#### Part 1: Identify Session Starts

**Goal:** Write a query that, for each event, determines if it is the start of a new session. An event is a "session start" if it's the user's very first event or if it occurred more than 30 minutes after their previous event.

**Skills Tested:**

  * **Window Function (`LAG`)**: To look back at a previous row.
  * **Timestamp Arithmetic**: To calculate the difference between events.
  * `CASE` statements for conditional logic.
  * `PARTITION BY` to operate within groups of user events.

#### Part 2: Assign a Unique Session ID

**Goal:** Using the "session start" flag from Part 1, assign a unique ID to each session. All events within the same session should have the same `session_id`. The ID should be unique per user (e.g., user 101's first session is 1, user 102's first session is 1, etc.).

**Skills Tested:**

  * **Advanced Window Functions**: A conditional cumulative `SUM`, a classic technique for solving "gaps and islands" problems.

#### Part 3: Summarize Each Session

**Goal:** With each event now tagged with a `session_id`, aggregate the data to produce a final report with one row per session, showing key metrics.

**Skills Tested:**

  * `GROUP BY` on multiple columns.
  * Aggregate functions (`MIN`, `MAX`, `COUNT`).
  * Putting it all together with **CTEs**.

-----

### Solution and Explanation

Here is the complete, multi-step query.

```sql
-- A CTE to calculate the time since the last event for each user
WITH TimeBetweenEvents AS (
    SELECT
        user_id,
        event_timestamp,
        event_type,
        -- Calculate the difference in minutes from the PREVIOUS event for the same user
        (
            EXTRACT(EPOCH FROM (event_timestamp - LAG(event_timestamp, 1) OVER (PARTITION BY user_id ORDER BY event_timestamp))) / 60
        ) AS minutes_since_last_event -- This syntax is for PostgreSQL. In MySQL, you'd use TIMESTAMPDIFF(MINUTE, LAG(...), event_timestamp)
    FROM events
),

-- A CTE to flag the start of each new session
SessionStarts AS (
    SELECT
        user_id,
        event_timestamp,
        event_type,
        -- An event is a session start if it's the first event (minutes_since_last_event is NULL)
        -- or if the time since the last one is > 30 minutes.
        CASE
            WHEN minutes_since_last_event IS NULL OR minutes_since_last_event > 30 THEN 1
            ELSE 0
        END AS is_session_start
    FROM TimeBetweenEvents
),

-- A CTE to assign a unique, incrementing session ID to each event
SessionIdentifiers AS (
    SELECT
        user_id,
        event_timestamp,
        event_type,
        -- The session ID is the cumulative sum of the 'is_session_start' flags.
        -- This brilliantly assigns a consistent ID to all events in a session.
        SUM(is_session_start) OVER (PARTITION BY user_id ORDER BY event_timestamp) AS session_id
    FROM SessionStarts
)

-- Final step: Aggregate the events into sessions to get the summary
SELECT
    user_id,
    session_id,
    MIN(event_timestamp) AS session_start_time,
    MAX(event_timestamp) AS session_end_time,
    -- Calculate session duration
    (
        EXTRACT(EPOCH FROM (MAX(event_timestamp) - MIN(event_timestamp))) / 60
    ) AS session_duration_minutes,
    COUNT(*) AS number_of_events
FROM SessionIdentifiers
GROUP BY
    user_id,
    session_id
ORDER BY
    user_id,
    session_id;

```

#### Expected Final Output:

| user\_id | session\_id | session\_start\_time  | session\_end\_time    | session\_duration\_minutes | number\_of\_events |
|---------|------------|---------------------|---------------------|--------------------------|------------------|
| 101     | 1          | 2025-09-07 10:00:00 | 2025-09-07 10:15:00 | 15.0                     | 3                |
| 101     | 2          | 2025-09-07 11:00:00 | 2025-09-07 11:02:00 | 2.0                      | 2                |
| 102     | 1          | 2025-09-07 12:00:00 | 2025-09-07 12:45:00 | 45.0                     | 3                |
| 103     | 1          | 2025-09-07 13:00:00 | 2025-09-07 13:00:00 | 0.0                      | 1                |

In [33]:
import time

def two_sum_brute_force(nums, target):
    n = len(nums)
    for i in range(n):
        for j in range(i + 1, n):
            if nums[i] + nums[j] == target:
                return [i, j]


# Example usage:
nums = [239,1,1004,2391,9]
target = 2400



start_time = time.perf_counter()
print(two_sum_brute_force(nums,target))
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")



def two_sum_optimal(nums2, target2):
    seen = {}  # Dictionary to store number -> index
    for i, num in enumerate(nums2):
        complement = target - num
        if complement in seen:
            return [seen[complement], i]
        seen[num] = i # Add the current number and its index to the map

# Example usage:
nums2 = [100,111,102,3, 2, 4,3,5,56,7,6,2,86,4,2,1,5,56,32,5,9,3,234,239,3, 2, 4,3,5,56,7,6,2,86,4,2,1,5,56,32,5,9,3,234,239,3, 2, 4,3,5,56,7,6,2,86,4,2,1,5,56,32,5,9,3,234,2390,3, 2, 4,3,5,56,7,6,2,86,4,2,1,5,56,32,5,9,3,234,239,3, 2, 4,3,5,56,7,6,2,86,4,2,1,5,56,32,5,9,3,234,2391,9]
target2 = 2400


start_time = time.perf_counter()
print(two_sum_optimal(nums2, target2))
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")



[3, 4]
Execution time: 0.000475 seconds
[104, 107]
Execution time: 0.000240 seconds
